# Bedrock with LangChain - Explain/Interpret a code snippet or program 
> *If you see errors, you may need to be allow-listed for the Bedrock models used by this notebook*

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show you how to explain or interpret a given code snippet or program.

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a framework for developing applications powered by language models. The key aspects of this framework allow us to augment the Large Language Models by chaining together various components to create advanced use cases.

In this notebook we will use the Bedrock API provided by LangChain. The prompt used in this example creates a custom LangChain prompt template for adding context to the code explain request. 

**Note:** *This notebook can be run within or outside of AWS environment.*

#### Context
In the previous example `01_sql_query_generation_w_bedrock.ipynb`, we explored how to use Bedrock API. In this notebook we will try to add a bit more complexity with the help of `PromptTemplates` to leverage the LangChain framework. `PrompTemplates` allow you to create generic shells which can be populated with information later and get model outputs based on different scenarios.

As part of this notebook we will explore the use of Amazon Bedrock integration within LangChain framework and how it could be used to generate or explain code with the help of `PromptTemplate`.

#### Pattern
We will simply provide the LangChain implementation of Amazon Bedrock API with an input consisting of a task, an instruction and an input for the model under the hood to generate an output without providing any additional example. The purpose here is to demonstrate how the powerful LLMs easily understand the task at hand and generate compelling outputs.

![](./images/code-interpret-langchain.png)

#### Use case
To demonstrate the code generation capability of models in Amazon Bedrock, let's take the use case of code explain.

#### Persona
You are Joe, a Java software developer, has been tasked to support a legacy C++ application for Vehicle Fleet Management. You need help to explain or interpret certain complex C++ code snippets as you are performing analyis to identify the business logic and potential problems with the code.

#### Implementation
To fulfill this use case, we will show you how you can Amazon Bedrock API with LangChain to explain C++ code snippets.


## Setup

In this notebook, we'll also install the [Hugging Face Transformers](https://huggingface.co/docs/transformers/index) library which we'll use for counting the number of tokens in an input prompt.

In [2]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.309 "transformers>=4.24,<5"

  Obtaining dependency information for boto3>=1.28.57 from https://files.pythonhosted.org/packages/c0/67/4d23a38313d7b37892a6d9c9260809f1a2f5a37feaf6f13da0aa27f57d6d/boto3-1.28.63-py3-none-any.whl.metadata
  Using cached boto3-1.28.63-py3-none-any.whl.metadata (6.7 kB)
  Obtaining dependency information for awscli>=1.29.57 from https://files.pythonhosted.org/packages/f0/12/6e567c34832d436c36fc96cef53e527e4e24700ba3a0fd6c3dbcb12a14a0/awscli-1.29.63-py3-none-any.whl.metadata
  Using cached awscli-1.29.63-py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for botocore>=1.31.57 from https://files.pythonhosted.org/packages/24/0e/39117ec73ea22e700503b3af1dbab270563d6d6ca862cf572899824e4212/botocore-1.31.63-py3-none-any.whl.metadata
  Using cached botocore-1.31.63-py3-none-any.whl.metadata (6.1 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Obtaining dependency information for s3transfer<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/5a/4b/fec9ce18f

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

In [6]:
import boto3
import json
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

bedrock_client = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True  # Default. Needed for invoke_model() from the data plane
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


## Invoke the Bedrock LLM Model

We'll begin with creating an instance of Bedrock class from llms. This expects a `model_id` which is the ARN of the model available in Amazon Bedrock. 

Optionally you can pass on a previously created boto3 client as well as some `model_kwargs` which can hold parameters such as `temperature`, `topP`, `maxTokenCount` or `stopSequences` (more on parameters can be explored in Amazon Bedrock console).

Available text generation models under Amazon Bedrock have the following IDs:

- amazon.titan-tg1-large
- ai21.j2-grande-instruct
- ai21.j2-jumbo-instruct
- anthropic.claude-instant-v1
- anthropic.claude-v2

Note that different models support different `model_kwargs`.

In [5]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v2",
                    client = bedrock_client, 
                    model_kwargs = inference_modifier 
                    )


## Create a LangChain custom prompt template

By creating a template for the prompt we can pass it different input variables to it on every run. This is useful when you have to generate content with different input variables that you may be fetching from a database.

In [6]:
# Vehicle Fleet Management Code written in C++
sample_code = """
#include <iostream>
#include <string>
#include <vector>

class Vehicle {
protected:
    std::string registrationNumber;
    int milesTraveled;
    int lastMaintenanceMile;

public:
    Vehicle(std::string regNum) : registrationNumber(regNum), milesTraveled(0), lastMaintenanceMile(0) {}

    virtual void addMiles(int miles) {
        milesTraveled += miles;
    }

    virtual void performMaintenance() {
        lastMaintenanceMile = milesTraveled;
        std::cout << "Maintenance performed for vehicle: " << registrationNumber << std::endl;
    }

    virtual void checkMaintenanceDue() {
        if ((milesTraveled - lastMaintenanceMile) > 10000) {
            std::cout << "Vehicle: " << registrationNumber << " needs maintenance!" << std::endl;
        } else {
            std::cout << "No maintenance required for vehicle: " << registrationNumber << std::endl;
        }
    }

    virtual void displayDetails() = 0;

    ~Vehicle() {
        std::cout << "Destructor for Vehicle" << std::endl;
    }
};

class Truck : public Vehicle {
    int capacityInTons;

public:
    Truck(std::string regNum, int capacity) : Vehicle(regNum), capacityInTons(capacity) {}

    void displayDetails() override {
        std::cout << "Truck with Registration Number: " << registrationNumber << ", Capacity: " << capacityInTons << " tons." << std::endl;
    }
};

class Car : public Vehicle {
    std::string model;

public:
    Car(std::string regNum, std::string carModel) : Vehicle(regNum), model(carModel) {}

    void displayDetails() override {
        std::cout << "Car with Registration Number: " << registrationNumber << ", Model: " << model << "." << std::endl;
    }
};

int main() {
    std::vector<Vehicle*> fleet;

    fleet.push_back(new Truck("XYZ1234", 20));
    fleet.push_back(new Car("ABC9876", "Sedan"));

    for (auto vehicle : fleet) {
        vehicle->displayDetails();
        vehicle->addMiles(10500);
        vehicle->checkMaintenanceDue();
        vehicle->performMaintenance();
        vehicle->checkMaintenanceDue();
    }

    for (auto vehicle : fleet) {
        delete vehicle; 
    }

    return 0;
}
"""

In [7]:
from langchain import PromptTemplate

# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["code", "programmingLanguage"], 
    template="""

Human: You will be acting as an expert software developer in {programmingLanguage}. 
You will explain the below code and highlight if there are any red flags or where best practices are not being followed.
<code>
{code}
</code>

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(code=sample_code, programmingLanguage="C++")


### Explain C++ Code for Vehicle Fleet management using Amazon Bedrock and LangChain

In [8]:
response = textgen_llm(prompt)

code_explanation = response[response.index('\n')+1:]

print_ww(code_explanation)


Overall, the code follows good object-oriented design principles using inheritance, polymorphism,
and abstraction. Creating a base Vehicle class with common attributes and methods that are
overridden in derived classes is a good approach.

Some positives:

- Using protected inheritance to allow derived classes access to base class members.

- Virtual methods in base class that are overridden in derived classes to achieve polymorphic
behavior.

- Smart pointers or RAII is used to avoid memory leaks by calling delete in main().

- The Vehicle class destructor is declared virtual to allow proper cleanup in derived classes.

Some areas for improvement:

- The Vehicle constructor could validate the registration number parameter to avoid bad data.

- Variables like capacityInTons in Truck could be validated to be positive.

- The addMiles() method could validate miles is positive before adding.

- Derived classes could override addMiles() and provide model-specific logic.

- performMaintena

## Summary

To conclude we learnt that invoking the LLM without any context might not yield the desired results. By adding context and further using the the prompt template to constrain the output from the LLM we are able to successfully get our desired output